# CodeSandbox Tensorflow Template

This is a starter template for Tensorflow. It's a great starting point for any Python, Machine Learning, or AI project in CodeSandbox.

This short introduction uses Keras to:

* Load a prebuilt dataset.
* Build a neural network machine learning model that classifies images.
* Train this neural network.
* Evaluate the accuracy of the model.

The code is based on the [TensorFlow 2 quickstart for beginners guide](https://www.tensorflow.org/tutorials/quickstart/beginner)

## Set up TensorFlow

Import TensorFlow into your program to get started:

In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

2024-01-24 17:05:13.183236: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-24 17:05:13.183295: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-24 17:05:13.184454: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-24 17:05:13.192958: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-24 17:05:14.168007: W tensorflow/compiler/tf2

TensorFlow version: 2.15.0


## Load a dataset

Load and prepare the MNIST dataset. The pixel values of the images range from 0 through 255. Scale these values to a range of 0 to 1 by dividing the values by `255.0`. This also converts the sample data from integers to floating-point numbers:

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11490434/11490434 [==============================] - 1s 0us/step


## Build a machine learning model

Build a `tf.keras.Sequential` model:

In [3]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

[`Sequential`](https://www.tensorflow.org/guide/keras/sequential_model) is useful for stacking layers where each layer has one input [tensor](https://www.tensorflow.org/guide/tensor) and one output tensor. Layers are functions with a known mathematical structure that can be reused and have trainable variables. Most TensorFlow models are composed of layers. This model uses the [`Flatten`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten), [`Dense`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense), and [`Dropout`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout) layers.

For each example, the model returns a vector of [logits](https://developers.google.com/machine-learning/glossary#logits) or [log-odds](https://developers.google.com/machine-learning/glossary#log-odds) scores, one for each class.

In [4]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.78725433, -0.935447  , -0.3716247 , -1.0335065 ,  0.5773967 ,
        -0.5598687 , -0.861437  ,  0.30536866, -0.0590751 ,  0.5143626 ]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to *probabilities* for each class: 

In [5]:
tf.nn.softmax(predictions).numpy()

array([[0.05267008, 0.04541556, 0.07981225, 0.04117351, 0.20616946,
        0.06611747, 0.04890427, 0.15706667, 0.10909583, 0.19357486]],
      dtype=float32)

Note: It is possible to bake the `tf.nn.softmax` function into the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to provide an exact and numerically stable loss calculation for all models when using a softmax output. 

Define a loss function for training using `losses.SparseCategoricalCrossentropy`:

In [6]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

The loss function takes a vector of ground truth values and a vector of logits and returns a scalar loss for each example. This loss is equal to the negative log probability of the true class: The loss is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10) ~= 2.3`.

In [7]:
loss_fn(y_train[:1], predictions).numpy()

2.7163224

Before you start training, configure and compile the model using Keras `Model.compile`. Set the [`optimizer`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) class to `adam`, set the `loss` to the `loss_fn` function you defined earlier, and specify a metric to be evaluated for the model by setting the `metrics` parameter to `accuracy`.

In [8]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## Train and evaluate your model

Use the `Model.fit` method to adjust your model parameters and minimize the loss: 

In [9]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5


2024-01-24 17:05:26.071323: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188160000 exceeds 10% of free system memory.


1875/1875 [==============================] - 4s 2ms/step - loss: 0.2973 - accuracy: 0.9132
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1415 - accuracy: 0.9583
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1093 - accuracy: 0.9672
Epoch 4/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0890 - accuracy: 0.9728
Epoch 5/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0772 - accuracy: 0.9760


The `Model.evaluate` method checks the model's performance, usually on a [validation set](https://developers.google.com/machine-learning/glossary#validation-set) or [test set](https://developers.google.com/machine-learning/glossary#test-set).

In [10]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - loss: 0.0772 - accuracy: 0.9763 - 589ms/epoch - 2ms/step


[0.0772414281964302, 0.9763000011444092]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [11]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [12]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[1.0166199e-08, 1.6718554e-10, 1.1981162e-05, 6.7399211e-05,
        2.5958166e-12, 5.6435541e-09, 9.1382709e-15, 9.9992013e-01,
        1.6571887e-09, 4.1566554e-07],
       [8.7262475e-07, 4.2979902e-04, 9.9949956e-01, 3.6019061e-05,
        1.5102008e-14, 1.1325332e-05, 3.9516944e-08, 1.3039589e-11,
        2.2422382e-05, 4.6378385e-13],
       [5.0779494e-07, 9.9418122e-01, 1.8627249e-04, 9.5610376e-05,
        2.6662383e-04, 1.6237364e-05, 1.4347377e-05, 4.8791664e-03,
        3.5846542e-04, 1.4164351e-06],
       [9.9996483e-01, 3.5040484e-11, 1.3404604e-05, 1.7727822e-07,
        9.5489789e-08, 7.0250502e-07, 6.2070485e-06, 1.3464590e-05,
        3.0609918e-08, 1.0358181e-06],
       [3.7389786e-06, 3.2861144e-10, 4.3004366e-06, 4.8077013e-08,
        9.9569327e-01, 3.0065291e-08, 5.8601781e-06, 3.0760429e-04,
        3.5292608e-06, 3.9815139e-03]], dtype=float32)>

## Conclusion

Congratulations! You have trained a machine learning model using a prebuilt dataset using the [Keras](https://www.tensorflow.org/guide/keras/overview) API.

For more examples of using Keras, check out the [tutorials](https://www.tensorflow.org/tutorials/keras/). To learn more about building models with Keras, read the [guides](https://www.tensorflow.org/guide/keras). If you want learn more about loading and preparing data, see the tutorials on [image data loading](https://www.tensorflow.org/tutorials/load_data/images) or [CSV data loading](https://www.tensorflow.org/tutorials/load_data/csv).

##### Copyright 2019 The TensorFlow Authors.

In [13]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.